In [29]:
## Start a new case, monitoting the web traffic
import requests
import lxml
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from datetime import datetime
import datetime
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
from getpass import getpass
import lxml
import logging
logging.basicConfig(filename='scraper.log', level=logging.INFO)

In [30]:
class h2s:
    '''
    
    '''

    def __init__(self, user_name:str, pass_word:str, host_name:str, name_database:str, port:int):
        '''
        user_name:str the user name of MySQL connection
        pass_word:str the password pf MySQL connection
        host_name:str the host name of this RDB
        port:str the number of host, should be a integer of 4

        Note that init method has no output which is just a intialization

        If you are working on your MySQL server, you need to use the connection below and which is the default value in __init__
        '''

        self.user_name = user_name
        self.pass_word = pass_word 
        self.host_name = host_name
        self.today = datetime.today()
        self.port = port
        self.name_database = name_database
        try:
            self.connection_str = f'mysql+pymysql://{user_name}:{pass_word}@{host_name}:{port}/{name_database}'
            self.sqlEngine = create_engine(self.connection_str)
            self.dbConnection = self.sqlEngine.connect()
            logging.info(f'{self.today}: connection establised')
        except:
            print(f'ERROR in server connection, please check')

    def h2sScraper (self,name_table):
        '''
        Note: If your are working with a MySQL server then use this
        '''

        self.name_table = name_table

        ## this is a generator of a random user for the sake of privacy
        ua = UserAgent()
        UserAgent_random = ua.random
        headers = {'User-Agent': UserAgent_random}

        ## this is the link to h2s
        page = requests.get(f'https://holland2stay.com/residences.html?&available_to_book=179&p={1}&product_list_limit=30', headers=headers)
        soup = BeautifulSoup(page.content,'lxml')

        ## I am too lazy to rename this variable, this a is a the class called regi-item d-flex flex-wrap, wraps all resources that can be book directly
        a = soup.find_all(class_ = 'regi-item d-flex flex-wrap')


        ## lists for dataframe below
        list_time = []
        list_street_num = []
        list_avalability = []
        list_city = []
        list_building = []
        list_furnishment = []
        list_num_floor = []
        list_size = []
        list_num_bedrooms = []
        list_occupancy = []
        list_contract = []
        list_stay = []


        for i in range(len(a)):

            soup = BeautifulSoup(str(a[i]), 'lxml')
            h4_element = soup.find('h4', class_='regularbold')
            li = soup.find_all('li')
            stree_num = h4_element.contents[0]
            info = [element.text.strip() for element in li]
            list_time.append(datetime.today())
            list_street_num.append(stree_num)
            list_avalability.append(info[0].replace('Available from',''))
            list_city.append(info[1])
            list_building.append(info[2])
            list_furnishment.append(info[3])
            list_num_floor.append(info[4]. replace('Floor ',''))
            list_size.append(info[5])
            list_num_bedrooms.append(info[6].replace('Bedrooms: ', ''))
            list_occupancy.append(info[7].replace('Max occupancy: ', ''))
            list_contract.append(info[8].replace('Contract type: ', ''))
            list_stay.append(info[9].replace('Minimum stay: ', ''))

        
        self.update = pd.DataFrame({"UpdateTime": list_time, "StreetNumber" : list_street_num, "Avalibility": list_avalability, "City" : list_city, "Building" : list_building,
                    "Furnishment" : list_furnishment, "NumberFloor" : list_num_floor, "RoomSize" : list_size, "BedroomNumber" : list_num_bedrooms,"MaxOccupancy":list_occupancy,
                    "ContractType" : list_contract, "MinimumStay" : list_stay})
        try:
            old = pd.read_sql(f'SELECT * FROM {self.name_database}.{self.name_table}', self.sqlEngine)
        except:
            old = pd.DataFrame({"UpdateTime": [], "StreetNumber" : [], "Avalibility": [], "City" : [], "Building" : [],
                    "Furnishment" : [], "NumberFloor" : [], "RoomSize" : [], "BedroomNumber" : [],"MaxOccupancy": [],
                    "ContractType" : [], "MinimumStay" : []})


        # Find missing rows, meaning to find what are already booked during time between this update and previous one
        missing_rows = old[~old.isin(self.update)].dropna()
        # Find new rows, meaning to find what are newly released
        new_rows = self.update[~self.update.isin(old)].dropna()

        ## export to MySQL as a temporary table
        self.update.to_sql(name_table,if_exists="replace",index=False ,con=self.dbConnection)
        return new_rows, missing_rows, self.update